У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [231]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score

In [195]:
# Завантажимо дані
df = pd.read_csv('customer_segmentation_train.csv')

In [196]:
df.dtypes

,0
ID,int64
Gender,object
Ever_Married,object
Age,int64
Graduated,object
Profession,object
Work_Experience,float64
Spending_Score,object
Family_Size,float64
Var_1,object


In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [198]:
7928/8068

0.9826474962816063

In [199]:
df.Ever_Married.unique()

array(['No', 'Yes', nan], dtype=object)

In [200]:
df.Graduated.unique()

array(['No', 'Yes', nan], dtype=object)

In [201]:
df.Profession.unique()

array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
       'Executive', 'Doctor', 'Homemaker', 'Marketing', nan], dtype=object)

In [202]:
df.Var_1.unique()

array(['Cat_4', 'Cat_6', 'Cat_7', 'Cat_3', 'Cat_1', 'Cat_2', nan, 'Cat_5'],
      dtype=object)

In [203]:
# Заповнюємо пропущені значення модою
for col in ['Ever_Married', 'Graduated', 'Profession', 'Var_1']:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)

/tmp/ipython-input-203-1514245690.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)


In [204]:
# Заповнюємо пропущені значення в 'Work_Experience' медіаною
median_work_experience = df['Work_Experience'].median()
df['Work_Experience'].fillna(median_work_experience, inplace=True)

/tmp/ipython-input-204-1251961531.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Work_Experience'].fillna(median_work_experience, inplace=True)


In [205]:
# Заповнюємо пропущені значення в 'Work_Experience' медіаною
median_work_experience = df['Work_Experience'].median()
df['Work_Experience'].fillna(median_work_experience, inplace=True)

/tmp/ipython-input-205-1251961531.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Work_Experience'].fillna(median_work_experience, inplace=True)


In [206]:
# Заповнюємо пропущені значення в 'Family_Size' медіаною
median_family_size = df['Family_Size'].median()
df['Family_Size'].fillna(median_family_size, inplace=True)

/tmp/ipython-input-206-284119154.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Family_Size'].fillna(median_family_size, inplace=True)


In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   object 
 5   Profession       8068 non-null   object 
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      8068 non-null   float64
 9   Var_1            8068 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


#### Кодування категоріальних ознак

In [208]:
df.select_dtypes(include='object')

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1,Segmentation
0,Male,No,No,Healthcare,Low,Cat_4,D
1,Female,Yes,Yes,Engineer,Average,Cat_4,A
2,Female,Yes,Yes,Engineer,Low,Cat_6,B
3,Male,Yes,Yes,Lawyer,High,Cat_6,B
4,Female,Yes,Yes,Entertainment,High,Cat_6,A
...,...,...,...,...,...,...,...
8063,Male,No,No,Artist,Low,Cat_1,D
8064,Male,No,No,Executive,Low,Cat_4,D
8065,Female,No,Yes,Healthcare,Low,Cat_6,D
8066,Female,No,Yes,Healthcare,Low,Cat_6,B


In [209]:
# Gender
gender_type_codes = {'Female': 0, 'Male': 1}
df['gender_type_codes'] = df.Gender.map(gender_type_codes)

In [210]:
# Ever_Married
ever_married_type_codes = {'No': 0, 'Yes': 1}
df['ever_married_type_codes'] = df.Ever_Married.map(ever_married_type_codes)

In [211]:
# Graduated
graduated_type_codes = {'No': 0, 'Yes': 1}
df['graduated_type_codes'] = df.Graduated.map(graduated_type_codes)

In [212]:
# Spending_Score
ordenc = OrdinalEncoder(categories=[['Low', 'Average', 'High']]) # визначаємо порядок категорій
ordenc.fit(df[['Spending_Score']])

df['Spending_Score_Codes'] = ordenc.transform(df[['Spending_Score']])

In [213]:
# Profession
column_to_encode = 'Profession'

enc = preprocessing.OneHotEncoder()
enc.fit(df[[column_to_encode]])

one_hot = enc.transform(df[[column_to_encode]]).toarray()

df[enc.categories_[0]] = one_hot

In [214]:
# Var_1
column_to_encode = 'Var_1'

enc = preprocessing.OneHotEncoder()
enc.fit(df[[column_to_encode]])

one_hot = enc.transform(df[[column_to_encode]]).toarray()

df[enc.categories_[0]] = one_hot

In [215]:
# цільовий стовпець Segmentation
le = LabelEncoder()

df['Segmentation'] = le.fit_transform(df['Segmentation'])

In [216]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,...,Homemaker,Lawyer,Marketing,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6,Cat_7
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,462643,Female,Yes,38,Yes,Engineer,1.0,Average,3.0,Cat_4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,462669,Female,Yes,40,Yes,Entertainment,1.0,High,6.0,Cat_6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Розбиття на тренувальну та тестувальну вибірки

In [217]:
columns_to_drop = ['ID', 'Segmentation', 'Gender', 'Ever_Married', 'Graduated',
                   'Profession', 'Spending_Score', 'Var_1']

In [218]:
inputs, targets = df.drop(columns=columns_to_drop, axis=1), df['Segmentation']

In [219]:
inputs.head()

,Age,Work_Experience,Family_Size,gender_type_codes,ever_married_type_codes,graduated_type_codes,Spending_Score_Codes,Artist,Doctor,Engineer,...,Homemaker,Lawyer,Marketing,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6,Cat_7
0,22,1.0,4.0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38,1.0,3.0,0,1,1,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,67,1.0,1.0,0,1,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,67,0.0,2.0,1,1,1,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,40,1.0,6.0,0,1,1,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [220]:
targets

,Segmentation
0,3
1,0
2,1
3,1
4,0
...,...
8063,3
8064,3
8065,3
8066,1


In [221]:
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42)

In [222]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6454 entries, 2821 to 7270
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      6454 non-null   int64  
 1   Work_Experience          6454 non-null   float64
 2   Family_Size              6454 non-null   float64
 3   gender_type_codes        6454 non-null   int64  
 4   ever_married_type_codes  6454 non-null   int64  
 5   graduated_type_codes     6454 non-null   int64  
 6   Spending_Score_Codes     6454 non-null   float64
 7   Artist                   6454 non-null   float64
 8   Doctor                   6454 non-null   float64
 9   Engineer                 6454 non-null   float64
 10  Entertainment            6454 non-null   float64
 11  Executive                6454 non-null   float64
 12  Healthcare               6454 non-null   float64
 13  Homemaker                6454 non-null   float64
 14  Lawyer                   6

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

#### SMOTENC

In [223]:
# Визначимо індекси категоріальних стовпців
categorical_cols_for_smotenc = ['gender_type_codes', 'ever_married_type_codes', 'graduated_type_codes', 'Spending_Score_Codes']
categorical_features_indices = [X_train.columns.get_loc(col) for col in categorical_cols_for_smotenc]

print("Назви категоріальних стовпців для SMOTENC:", categorical_cols_for_smotenc)
print("Індекси категоріальних стовпців для SMOTENC:", categorical_features_indices)


Назви категоріальних стовпців для SMOTENC: ['gender_type_codes', 'ever_married_type_codes', 'graduated_type_codes', 'Spending_Score_Codes']
Індекси категоріальних стовпців для SMOTENC: [3, 4, 5, 6]


In [224]:
smotenc = SMOTENC(categorical_features=categorical_features_indices, random_state=42)
smotenc

SMOTENC(categorical_features=[3, 4, 5, 6], random_state=42)

In [225]:
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

print("\n--- Результати SMOTENC ---")
print(f"Розмір тренувальної вибірки до ресемплінгу: {X_train.shape}")
print(f"Розмір тренувальної вибірки після SMOTENC: {X_train_smotenc.shape}")
print("\nРозподіл класів до SMOTENC:")
print(y_train.value_counts())
print("\nРозподіл класів після SMOTENC:")
print(y_train_smotenc.value_counts())


--- Результати SMOTENC ---
Розмір тренувальної вибірки до ресемплінгу: (6454, 23)
Розмір тренувальної вибірки після SMOTENC: (7176, 23)

Розподіл класів до SMOTENC:
Segmentation
3    1794
2    1590
0    1581
1    1489
Name: count, dtype: int64

Розподіл класів після SMOTENC:
Segmentation
0    1794
3    1794
1    1794
2    1794
Name: count, dtype: int64


#### SMOTE-Tomek

In [226]:
smote_tomek = SMOTETomek(random_state=42)

X_resampled_smotetomek, y_resampled_smotetomek = smote_tomek.fit_resample(X_train, y_train)

print("\n--- Результати SMOTE-Tomek ---")
print("Розмір X_train до SMOTE-Tomek:", X_train.shape)
print("Розмір X_resampled_smotetomek після SMOTE-Tomek:", X_resampled_smotetomek.shape)
print("\nРозподіл класів y_train до SMOTE-Tomek:\n", y_train.value_counts())
print("\nРозподіл класів y_resampled_smotetomek після SMOTE-Tomek:\n", y_resampled_smotetomek.value_counts())


--- Результати SMOTE-Tomek ---
Розмір X_train до SMOTE-Tomek: (6454, 23)
Розмір X_resampled_smotetomek після SMOTE-Tomek: (6418, 23)

Розподіл класів y_train до SMOTE-Tomek:
 Segmentation
3    1794
2    1590
0    1581
1    1489
Name: count, dtype: int64

Розподіл класів y_resampled_smotetomek після SMOTE-Tomek:
 Segmentation
2    1637
3    1609
1    1603
0    1569
Name: count, dtype: int64


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

#### Навчання моделі на оригінальних даних

In [227]:
model_original = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


#### Навчання моделі на даних, збалансованих SMOTENC

In [228]:
model_smotenc = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)
model_smotenc.fit(X_train_smotenc, y_train_smotenc)
y_pred_smotenc = model_smotenc.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


#### Модель на даних, збалансованих SMOTE-Tomek

In [229]:
model_smotetomek = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)
model_smotetomek.fit(X_resampled_smotetomek, y_resampled_smotetomek)
y_pred_smotetomek = model_smotetomek.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [230]:
# --- Звіт для оригінальних даних ---
print("\n" + "="*50)
print("Classification Report - Оригінальні дані")
print("="*50)
print(classification_report(y_test, y_pred_original))

# --- Звіт для даних SMOTENC ---
print("\n" + "="*50)
print("Classification Report - Дані SMOTENC")
print("="*50)
print(classification_report(y_test, y_pred_smotenc))

# --- Звіт для даних SMOTE-Tomek ---
print("\n" + "="*50)
print("Classification Report - Дані SMOTE-Tomek")
print("="*50)
print(classification_report(y_test, y_pred_smotetomek))


Classification Report - Оригінальні дані
              precision    recall  f1-score   support

           0       0.41      0.49      0.44       391
           1       0.43      0.12      0.18       369
           2       0.45      0.65      0.53       380
           3       0.66      0.70      0.68       474

    accuracy                           0.50      1614
   macro avg       0.49      0.49      0.46      1614
weighted avg       0.50      0.50      0.48      1614


Classification Report - Дані SMOTENC
              precision    recall  f1-score   support

           0       0.42      0.50      0.46       391
           1       0.42      0.19      0.26       369
           2       0.46      0.63      0.53       380
           3       0.69      0.67      0.68       474

    accuracy                           0.51      1614
   macro avg       0.50      0.50      0.48      1614
weighted avg       0.51      0.51      0.50      1614


Classification Report - Дані SMOTE-Tomek
        

#### Напишіть, яку метрику ви обрали для порівняння моделей.

Для порівняння моделей в умовах незбалансованої багатокласової класифікації найкраще зосередитися на таких метриках, як макросередній `macro avg F1-score` або середня точність / `accuracy`. Але, якщо правильно розумію, то з `accuracy` треба бути обережним якщо класи сильно незбалансовани. В нашому завданні наче це не так, сильного дизбалансу не спостерігається. Але зверну увагу більше на `macro avg F1-score`, він здається більш надійним, бо він обчислює F1-score для кожного класу, а потім бере просте середнє значення.

#### Яка модель найкраща?

`macro avg F1-score` у нас наступні:

- Оригінальні дані: macro avg f1-score = 0.46

- Дані SMOTENC: macro avg f1-score = 0.48

- Дані SMOTE-Tomek: macro avg f1-score = 0.47

На основі цієї метрики, модель, навчена на даних, збалансованих за допомогою SMOTENC, є найкращою.

#### Якщо немає суттєвої різниці - гіпотеза, чому?

Різниця між моделями не є драматичною.

Мабуть це може бути через якімть ознак моделі: існуючі ознаки можуть бути недостатньо інформативними, щоб дозволити логістичній регресії добре розділити класи. Можливо, в даних присутні складні взаємодії між ознаками, які логістична регресія не може уловити. Ресемплінг допомагає збалансувати розподіл, але не покращує якість самих ознак.

Або

Проблема може бути в самій моделі: логістична регресія намагається знайти лінійну межу, яка розділяє класи. Якщо дані є нелінійно розділеними, то навіть додавання синтетичних зразків може не допомогти моделі знайти кращу лінійну межу.